# Task Description

In [1]:
# The task would be to test a long-only stock-based carry/value strategy.

# Carry strategy in stocks would be to long high dividend yield stocks and short the reverse (in this case no need to do short).
# Whole value strategy you should be familiar with.

# Carry trade in rate hike periods work pretty well (for fx especially). We are interested in whether the same hold for value. 
# The underlying assets would be S&P 500 stocks.

# Please take into account that some companies do stock repurchase instead of cash dividend.
# You could try to find adjusted dividend yield data. Time horizon is up to you, but keep in mind potential survivorship bias.
# It might be interesting to check the strategy performance during different periods. 
# Remember to make the backtest a way that they could be implemented (not longing 100 stocks at the same time). 
# Have a nice night and good luck with your presentation on Monday.

# Setup

## Load Packages

In [2]:
#### Import required Packages ####
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import scipy as sp
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error # to calculate the MSE
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score

from statsmodels.graphics.tsaplots import plot_acf # To produce ACF plots
from statsmodels.graphics.tsaplots import plot_pacf # To produce PACF plots
from statsmodels.tsa.seasonal import seasonal_decompose # To decompose Seasons
from statsmodels.tsa.stattools import adfuller, kpss # Tests for Stationarity
from statsmodels.tsa.ar_model import AutoReg # To produce AR models
from statsmodels.stats.anova import anova_lm # To use ANOVA (compare nested models)
from statsmodels.tsa.arima.model import ARIMA # To build ARMA & ARIMA Models
import statsmodels.stats.diagnostic as dg # To get Breusch-Godfrey Test
from statsmodels.stats.stattools import durbin_watson



from datetime import datetime # to transform variables into datetime objects
import math # simple math functions
from math import sqrt # square root function
import statistics # descriptive statistics library
import scipy.stats as stats # descriptive statistics library from scipy
import matplotlib.dates as mdates # date formatting
from matplotlib.collections import PolyCollection, LineCollection # better plot options

C:\Users\Corinne Vogel\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Corinne Vogel\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Corinne Vogel\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Corinne Vogel\anaconda3\lib\site-packages\statsmodels\ts

## Useful functions for Matrix calculations

In [3]:
def f(x, y): 
    return x + y                             
vecsum = np.vectorize(f)     # Use pd.DataFrame(vecsum(A,B)) to get df of elementwise sum

In [4]:
def g(x, y): 
    return x - y                             
vecdif = np.vectorize(g)     # Use pd.DataFrame(vecdif(A,B)) to get df of elementwise differences

In [5]:
def h(x, y): 
    return x * y                             
vecmult = np.vectorize(h)    # Use pd.DataFrame(vecmult(A,B)) to get df of elementwise multiplication

In [6]:
def i(x, y): 
    return x / y                             
vecdiv = np.vectorize(i)    # Use pd.DataFrame(vecdiv(A,B)) to get df of elementwise division

## Plotstyle

In [7]:
# Seaborn plot style ticks to have nicer looking plots
sb.set_style("ticks")
sb.mpl.rc("figure", figsize=(16,8))
sb.mpl.rc("font", size=14)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

## LoadData

In [ ]:
# Loading Data

data_px = pd.read_excel("02_Data_clean/SPX_px_cleanest.xlsx", parse_dates=["Date"])
data_px_yly = pd.read_excel("02_Data_clean/SPX_px_clean_yearly.xlsx", parse_dates=["Date"])
data_bv = pd.read_excel("02_Data_clean/SPX_value_clean.xlsx", parse_dates=["Date"])

bv_resample_fail = pd.read_excel("02_Data_clean/SPX_btm_resample_fail_clean.xlsx")

# Check
print(type(data_bv))
print(data_bv.dtypes)

## Data Cleaning

In [ ]:
# Resample monthly returns
data_px.set_index("Date", inplace=True) # set datetimeindex
data_bv.set_index("Date", inplace=True) # set datetimeindex
data_px_yly.set_index("Date", inplace=True) # set datetimeindex


In [ ]:
# Create monthly returns
mtl_ret = data_px.pct_change(fill_method = None).resample("M").agg(lambda x: ((1+x).prod()-1)) # pct_change creates ordinary returns

In [ ]:
mtl_ret = mtl_ret.loc["1999-12-31":"2022-06-30", :]
mtl_ret

### Create Book to Market Dataframe

In [ ]:
# Divide BVPS yearly by SP yearly to get Book to market signal yearly
data_btm = pd.DataFrame(vecdiv(data_bv, data_px_yly))

In [ ]:
# Set Datetimeindex again
data_btm.set_index(data_px_yly.index, inplace=True) # set datetimeindex

### Resample BTM for monthly rebalance

In [ ]:
data_btm_mt = data_btm.resample("M").ffill()

In [ ]:
data_btm_mt

In [ ]:
bv_resample_fail.set_index("Date", inplace=True)


In [ ]:
# data_btm_mt = data_btm_mt.append(bv_resample_fail) # Months until June 2022 added bc resample is a bitch

In [ ]:
data_btm_mt = pd.concat(
    [data_btm_mt, bv_resample_fail],
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)

In [ ]:
data_btm_mt

# Strategy

## Calculate Desired Weights

In [ ]:
# The approach is as follows: 
# 1. Take Book to market value as BVPS(t-6mt) / SP(t-6mt)
# 2. for each month calculate the 10 stocks considered for the long strategy
# 3. calculate the PF return
# 4. take into account the transaction costs

In [ ]:
np.shape(data_btm_mt)

In [ ]:
# Create empty df for weights
cols = (data_btm_mt.columns[0:])
rows = (data_btm_mt.iloc[:,0])

des_weight = pd.DataFrame(index = rows, columns = (cols))
des_weight.set_index(data_btm_mt.index, inplace=True) # set datetimeindex
des_weight

In [ ]:
data_btm_mt.index = data_btm_mt.index.astype('datetime64[ns]')

In [ ]:
data_btm_mt.index.dtype

In [ ]:
# Assign weights based on 10 largest BTM value
for i in data_btm_mt.index:
    x = data_btm_mt.loc[i,:].nlargest(10)
    for j in data_btm_mt.columns:
        if j in x.index:
            des_weight.loc[i,j] = 0.1
        else:
            des_weight.loc[i,j] = 0

In [ ]:
des_weight.head(26)

## Weighted Returns on 30. June

In [ ]:
# Cut for analysis dataframe
mtl_ret = mtl_ret.loc["2000-06-30":"2022-06-30", :]
des_weight = des_weight.loc["2000-06-30":"2022-06-30", :]

In [ ]:
weighted_ret = pd.DataFrame(vecmult(des_weight, mtl_ret))

In [ ]:
weighted_ret.set_index(mtl_ret.index, inplace=True) # set datetimeindex
weighted_ret

## PF returns weighted per year

In [ ]:
ones500x1 = np.ones(len(weighted_ret.columns))
pd.DataFrame(ones500x1)

In [ ]:
pf_ret_unadj = np.matmul(weighted_ret,ones500x1)
pf_ret_unadj = pd.DataFrame(pf_ret_unadj)
pf_ret_unadj.columns =["PF_returns_unadjusted"]
pf_ret_unadj.loc["2000-06-30"] = 0.0 #bc we do not make returns in first period
pf_ret_unadj

## Actual weights

In [ ]:
# 1+return -> for stock returns and for the weighted PF returns -> used for actual weights calculation
mtl_ret_gross = mtl_ret.transform(lambda x: x + 1)
pf_ret_unadj_gross = pf_ret_unadj.transform(lambda x: x + 1)
mtl_ret_gross


In [ ]:
# Create (23x500) matrix to match midyear return matrix for scaling factor to calculate adjusted weights
ones_matrix = np.ones((len(pf_ret_unadj_gross),500))
pf_ret_unadj_gross_matrix = pd.DataFrame(vecmult(pf_ret_unadj_gross, ones_matrix))
pf_ret_unadj_gross_matrix

In [ ]:
# Calculate weight scaling factor by deviding the returns of the assets, per year by the weighted pf return per year
weight_scaleFactor = pd.DataFrame(vecdiv(mtl_ret_gross,pf_ret_unadj_gross_matrix), columns = mtl_ret_gross.columns, index = mtl_ret_gross.index)

In [ ]:
des_weight

In [ ]:
# Shift by -1 to create w(t-1) but last row becomes N/A
des_weight_lag1 = des_weight.shift(1)

In [ ]:
# Calculate actual weights before rebalancing
actual_weight = pd.DataFrame(vecmult(des_weight_lag1, weight_scaleFactor), columns = weight_scaleFactor.columns, index = weight_scaleFactor.index)
actual_weight

## Turnover

In [ ]:
des_weight

In [ ]:
actual_weight.head(25)

In [ ]:
turnover = pd.DataFrame(abs(vecdif(des_weight, actual_weight)), index = weight_scaleFactor.index)
turnover.loc["2000-06-30",:] = des_weight.loc["2000-06-30",:] # Adjust for buying the whole portfolio at the 30.06.2000
turnover

In [ ]:
turnover_mtl = np.matmul(turnover,ones500x1)
turnover_mtl = pd.DataFrame(turnover_mtl, columns = ["Yearly Turnover of PF"])
turnover_mtl.head(40)

# Adjust Portfolio Return for Transaction cost and turnover

In [ ]:
# Calculate transaction costs yearly by multiplying turnover and transaction costs
c = np.ones(len(turnover_mtl)) * 0.002 # 0.2% from Michael Milliker for S&P500 stocks -> transaction cost corresponding to turnover in %
c = pd.DataFrame(c)
tc_mtl = pd.DataFrame(vecmult(turnover_mtl,c), index=turnover_mtl.index, columns= ["TC_mtl"])
tc_mtl

In [ ]:
pf_ret_unadj

In [ ]:
pf_ret_adj = pd.DataFrame(vecdif(pf_ret_unadj, tc_mtl), index = tc_mtl.index, columns = ["PF_ret_adj"])
pf_ret_adj

# Plot Results

In [ ]:
pf_ret_adj_gross = pf_ret_adj.transform(lambda x: x + 1)
pf_ret_adj_gross_cum = pf_ret_adj_gross.cumprod() # We don't start at one bc of initial costs for buying pf
pf_ret_unadj_gross_cum = pf_ret_unadj_gross.cumprod() # We don't start at one bc of initial costs for buying pf
pf_ret_adj_gross_cum.columns = ["pf_ret_adj_cum"]
pf_ret_adj_gross

In [ ]:
fig, ax = plt.subplots()
ax.plot(pf_ret_adj_gross_cum.index,pf_ret_adj_gross_cum, label="Value Long Leg with TC", color="black")
ax.plot(pf_ret_unadj_gross_cum.index,pf_ret_unadj_gross_cum, label="Value Long Leg no TC", color="grey")
ax.legend(loc="best")
ax.set_ylabel('Cumulative Returns, indexed 30.06.2000')
plt.savefig("03_Figures/Value_Performance_monthly_rebalance")

In [ ]:
fig, ax = plt.subplots()
ax.plot(pf_ret_adj.index,pf_ret_adj, label="Value Long Leg monthly rebalance with TC", color="black")
ax.legend(loc="best")
ax.set_ylabel('Monthly Adj. PF Returns, indexed 30.06.2000')
plt.savefig("03_Figures/Value_monthly_returns")

In [ ]:
pf_ret_adj

In [ ]:
x = [i for i in pf_ret_adj.iloc[:,0] if i > 0.05]
len(x)

In [ ]:
x = [i for i in pf_ret_adj.iloc[:,0] if i < -0.05]
len(x)

In [ ]:

# mtl_ret_describe = mtl_ret.describe()
# mtl_ret_describe.to_excel("mlt_ret_describe.xlsx")

# pf_ret_adj.to_excel("pf_ret_adj.xlsx")

# des_weight.columns = [mtl_ret.columns]
# des_weight.to_excel("des_weights.xlsx")
# mtl_ret.to_excel("mtl_ret.xlsx")

dtypes = data_px.dtypes
dtypes.to_excel("dtypes.xlsx")

# Performance Statistics

In [ ]:
pf_ret_adj.describe()

In [ ]:
pf_ret_adj.mean()

# Comparison with Benchmark & FED Fund Rate

In [ ]:
data_spx = pd.read_excel("02_Data_clean/SPX_Index_clean.xlsx", parse_dates=["Date"])
data_fund = pd.read_excel("02_Data_clean/FED_Fund_rate_Upper_bound.xlsx", parse_dates=["Date"])
data_spx.set_index("Date", inplace=True) # set datetimeindex

In [ ]:
spx_gross = data_spx.transform(lambda x: x + 1)
spx_gross_cum = spx_gross.cumprod() 

In [ ]:
spx_gross_cum.head(20)

In [ ]:
pf_ret_adj_gross

In [ ]:
pf_ret_adj_gross_cum

In [ ]:
fig, ax = plt.subplots()
ax.plot(pf_ret_adj_gross_cum.index,pf_ret_adj_gross_cum, label="Value Long Leg with TC", color="black")
ax.plot(spx_gross_cum.index,spx_gross_cum, label="SPX Index (Survivors)", color="grey")
ax.legend(loc="best")
ax.set_ylabel('Cumulative Returns, indexed 30.06.2000')
plt.savefig("03_Figures/Value_Performance_mtl_rbl_index_comparison")